Konfiguracja i połączenie

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os 
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder \
    .appName("Koledzy") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33") \
    .config("spark.python.worker.reuse", "false") \
    .getOrCreate()

db_url = "jdbc:mysql://localhost:3306/grupa?useSSL=false"
db_properties = {
    "user": "root",
    "password": "",
    "driver": "com.mysql.cj.jdbc.Driver"
}

Pobieranie danych

In [2]:
df_koledzy = spark.read.jdbc(url=db_url, table="osoby", properties=db_properties)
df_koledzy.show()

+---+-------+----------+-------------------+---------+----+----+--------------+-----------+
| id|   imie|  nazwisko|              email|  telefon|plec|wiek|        miasto|technologia|
+---+-------+----------+-------------------+---------+----+----+--------------+-----------+
|  1| Daniel|    Cachro|    cachro@mail.com|123456789|   M|  24|Czarny Dunajec|     Python|
|  2|  Dawid|    Morawa|    morawa@mail.com|123456789|   M|  23|     Nowy Targ|         JS|
|  3|Dominik|Spytkowski|spytkowski@mail.com|123456789|   M|  23|     Nowy Targ|        C++|
|  4|Jędrzej| Pawłowski| pawlowski@mail.com|123456789|   M|  23|   Kościelisko|         C#|
|  5| Kacper|   Siciarz|   siciarz@mail.com|123456789|   M|  24|      Sieniawa|        CSS|
|  6|  Jakub|     Bąbol|     babol@mail.com|123456789|   M|  23|   Rabka-Zdrój|         JS|
|  7| Maciek|  Palenica|  palenica@mail.com|123456789|   M|  23|      Zakopane|       HTLM|
|  8|  Marta|   Stuglik|   stuglik@mail.com|123456789|   K|  23|Czarny Dunajec| 

Osoby starsze niz 22 lata

In [3]:
df_starsi = df_koledzy.filter(col("wiek") > 22)
df_starsi.show()

+---+-------+----------+-------------------+---------+----+----+--------------+-----------+
| id|   imie|  nazwisko|              email|  telefon|plec|wiek|        miasto|technologia|
+---+-------+----------+-------------------+---------+----+----+--------------+-----------+
|  1| Daniel|    Cachro|    cachro@mail.com|123456789|   M|  24|Czarny Dunajec|     Python|
|  2|  Dawid|    Morawa|    morawa@mail.com|123456789|   M|  23|     Nowy Targ|         JS|
|  3|Dominik|Spytkowski|spytkowski@mail.com|123456789|   M|  23|     Nowy Targ|        C++|
|  4|Jędrzej| Pawłowski| pawlowski@mail.com|123456789|   M|  23|   Kościelisko|         C#|
|  5| Kacper|   Siciarz|   siciarz@mail.com|123456789|   M|  24|      Sieniawa|        CSS|
|  6|  Jakub|     Bąbol|     babol@mail.com|123456789|   M|  23|   Rabka-Zdrój|         JS|
|  7| Maciek|  Palenica|  palenica@mail.com|123456789|   M|  23|      Zakopane|       HTLM|
|  8|  Marta|   Stuglik|   stuglik@mail.com|123456789|   K|  23|Czarny Dunajec| 

Dodawanie dodatkowej osoby

In [4]:
nowe_dane = [
    ("Dariusz", "Gał", "gal@mail.com", "123456789", "M", 23, "Czarny Dunajec", "React")
]
kolumny = ["imie", "nazwisko", "email", "telefon", "plec", "wiek", "miasto", "technologia"]

df_nowa_osoba = spark.createDataFrame(nowe_dane, schema=kolumny)
try:
    df_nowa_osoba.write.jdbc(
        url=db_url,
        table="osoby",
        mode="append",
        properties=db_properties
    )
    print("Dodano nowa osobe")
except Exception as e:
    print(f"Blad: {e}")

spark.read.jdbc(url=db_url, table="osoby", properties=db_properties).show()

Dodano nowa osobe
+---+-------+----------+-------------------+---------+----+----+--------------+-----------+
| id|   imie|  nazwisko|              email|  telefon|plec|wiek|        miasto|technologia|
+---+-------+----------+-------------------+---------+----+----+--------------+-----------+
|  1| Daniel|    Cachro|    cachro@mail.com|123456789|   M|  24|Czarny Dunajec|     Python|
|  2|  Dawid|    Morawa|    morawa@mail.com|123456789|   M|  23|     Nowy Targ|         JS|
|  3|Dominik|Spytkowski|spytkowski@mail.com|123456789|   M|  23|     Nowy Targ|        C++|
|  4|Jędrzej| Pawłowski| pawlowski@mail.com|123456789|   M|  23|   Kościelisko|         C#|
|  5| Kacper|   Siciarz|   siciarz@mail.com|123456789|   M|  24|      Sieniawa|        CSS|
|  6|  Jakub|     Bąbol|     babol@mail.com|123456789|   M|  23|   Rabka-Zdrój|         JS|
|  7| Maciek|  Palenica|  palenica@mail.com|123456789|   M|  23|      Zakopane|       HTLM|
|  8|  Marta|   Stuglik|   stuglik@mail.com|123456789|   K|  2